#NLP 18th Jan
##Roll no. J014
###SMS Classification


In [0]:
# import rasa_core
from nltk.tokenize import word_tokenize
from nltk.tokenize.treebank import TreebankWordDetokenizer

In [2]:
!wget 'https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip'

--2020-01-19 14:27:24--  https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 203415 (199K) [application/x-httpd-php]
Saving to: ‘smsspamcollection.zip’

smsspamcollection.z 100%[===================>] 198.65K   469KB/s    in 0.4s    

2020-01-19 14:27:31 (469 KB/s) - ‘smsspamcollection.zip’ saved [203415/203415]



In [3]:
!unzip -o smsspamcollection.zip

Archive:  smsspamcollection.zip
  inflating: SMSSpamCollection       
  inflating: readme                  


In [4]:
!ls

readme	sample_data  SMSSpamCollection	smsspamcollection.zip


In [0]:
import pandas as pd
from nltk.corpus import stopwords

In [6]:
import re
import nltk
nltk.download(['punkt','stopwords'])


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
data=pd.read_csv('/content/SMSSpamCollection',delimiter='\t',header=None)

In [0]:
#renaming columns
data.columns=['category','text']

In [9]:
data.head()

,category,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [0]:
#setting stopwords
sw=set(stopwords.words('english'))

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [0]:
#function for preprocessing
def word_preprocessor(text):
  text=re.sub('[^a-zA-Z]', " ", str(text))
  text=text.lower()
  text_tokens=word_tokenize(text)
  text_tokens_no_sw=[w for w in text_tokens if not w in sw]
  text=TreebankWordDetokenizer().detokenize(text_tokens_no_sw)
  return text

In [0]:
data['processed_text']=data['text'].apply(word_preprocessor)

In [0]:
#dropping original text cols
data.drop(['text'],1,inplace=True)

In [16]:
data['processed_text'][1]

'ok lar joking wif u oni'

In [0]:
#creating TfidfVectorizer object
model=TfidfVectorizer()

In [0]:
score=model.fit_transform(data['processed_text'])

In [0]:
dense=score.todense()
dense1=dense.tolist()

In [0]:
col_names=model.get_feature_names()

In [0]:
#creating a new df of scores and feature_names
df = pd.DataFrame(dense1,columns = col_names)

In [0]:
import numpy as np

In [0]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split as tts
from sklearn.metrics import classification_report,accuracy_score,confusion_matrix

In [24]:
data.head()

,category,processed_text
0,ham,go jurong point crazy available bugis n great ...
1,ham,ok lar joking wif u oni
2,spam,free entry wkly comp win fa cup final tkts st ...
3,ham,u dun say early hor u c already say
4,ham,nah think goes usf lives around though


In [0]:
df1=pd.DataFrame(data['category'])

In [26]:
df1.head()

,category
0,ham
1,ham
2,spam
3,ham
4,ham


In [0]:
combined_df=pd.concat([df,df1],1)

In [28]:
combined_df.head()

,aa,aah,aaniye,aaooooright,aathi,ab,abbey,abdomen,abeg,abel,aberdeen,abi,ability,abiola,abj,able,abnormally,aboutas,abroad,absence,absolutely,absolutly,abstract,abt,abta,aburo,abuse,abusers,ac,academic,acc,accent,accenture,accept,access,accessible,accidant,accident,accidentally,accommodation,...,young,younger,youphone,youre,yourinclusive,yourjob,youuuuu,youwanna,yoville,yowifes,yoyyooo,yr,yrs,ystrday,yt,ything,yummmm,yummy,yun,yunny,yuo,yuou,yup,yupz,zac,zaher,zealand,zebra,zed,zeros,zf,zhong,zindgi,zoe,zogtorius,zoom,zouk,zs,zyada,category
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,ham
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,ham
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,spam
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,ham
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,ham


In [29]:
combined_df['category'].isna().sum()

0

In [0]:
#tts
x_tr,x_te,y_tr,y_te=tts(combined_df.drop(['category'],1),combined_df['category'],test_size=0.2)

In [0]:
#initializing RFC
rfc=RandomForestClassifier()

In [0]:
model1=rfc.fit(x_tr,y_tr)

In [0]:
modelpreds=model1.predict(x_te)

In [34]:
accuracy_score(y_te,modelpreds)

0.9695067264573991

In [35]:
print(classification_report(y_te,modelpreds))

              precision    recall  f1-score   support

         ham       0.97      1.00      0.98       971
        spam       1.00      0.76      0.87       144

    accuracy                           0.97      1115
   macro avg       0.98      0.88      0.92      1115
weighted avg       0.97      0.97      0.97      1115



In [36]:
confusion_matrix(y_te,modelpreds)

array([[971,   0],
       [ 34, 110]])